In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

import gc

In [2]:
import pickle

In [3]:
filename_jan = 'data/fhv_tripdata_2021-01.parquet'

In [4]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df["duration"] = df.dropOff_datetime - df.pickup_datetime
    df["duration"]= df.duration.apply(lambda td: td.total_seconds() / 60)

    categorical = ["pickup_datetime", "dropOff_datetime"]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe(filename_jan)

In [6]:
len(df_train)

1154112

In [7]:
df_train["duration"].mean()

19.1672240937939

In [8]:
df_train["duration"].describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [9]:
df_train = df_train[((df_train.duration >= 1 ) & (df_train.duration <= 60 ))]

In [10]:
len(df_train)

1109826

In [11]:
df_train.isnull().mean() * 100

dispatching_base_num        0.000000
pickup_datetime             0.000000
dropOff_datetime            0.000000
PUlocationID               83.527328
DOlocationID               13.327044
SR_Flag                   100.000000
Affiliated_base_number      0.069651
duration                    0.000000
dtype: float64

In [12]:
df_train["PUlocationID"] = df_train["PUlocationID"].fillna(value=-1)
df_train["DOlocationID"] = df_train["DOlocationID"].fillna(value=-1)

In [13]:
df_train.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [14]:
len(df_train)

1109826

In [15]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df["duration"] = df.dropOff_datetime - df.pickup_datetime
    df["duration"]= df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 1 ) & (df.duration <= 60 ))]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)

    df["PUlocationID"] = df["PUlocationID"].fillna(value=-1)
    #df["DOlocationID"] = df["DOlocationID"].fillna(value=-1)
    
    return df

In [16]:
filename_jan = 'data/fhv_tripdata_2021-01.parquet'
filename_fev = 'data/fhv_tripdata_2021-02.parquet'

In [17]:
df_train = read_dataframe(filename_jan)
df_val = read_dataframe(filename_fev)

In [18]:
categorical = ['PUlocationID', 'DOlocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [22]:
mean_squared_error(y_train, y_pred, squared=False)

10.528519107203781

In [23]:
y_pred_val = lr.predict(X_val)

In [24]:
mean_squared_error(y_val, y_pred_val, squared=False)

11.014283141597323